In [308]:
import cv2
import numpy as np
import os

In [309]:
"""Function draws a line based on two points - upper pants corner and lower pants corner"""
def build_line(s,e,h):
    k = (s[0] - e[0]) / (s[1] - e[1])
    b = e[0] - k*s[0]

    return [round(h*k+b), h]

In [310]:
"""Function finds segment for pants/skirts. Sometimes segments are blue, sometimes are red"""
def find_pants(seg):
    # some pants are blue, some are read, need to check that
    pants_red = cv2.inRange(seg, (0, 0, 190), (0, 0, 195))
    pants_blue = cv2.inRange(seg, (120, 0, 60), (130, 0, 65))
    _, counts_red = np.unique(pants_red, return_counts=True)
    _, counts_blue = np.unique(pants_blue, return_counts=True)
    try:
        red = counts_red[1]
    except IndexError:
        red = 0
    try:
        blue = counts_blue[1]
    except IndexError:
        blue = 0

    if red>=blue:
        skr = pants_red
    else:
        skr = pants_blue

    return skr

In [311]:
"""Function finds corners of segment with Harris algorithm"""
def get_corners(img):
    dst = cv2.cornerHarris(img, 2, 3, 0.04)
    mask = np.zeros_like(img)
    mask[dst>0.01*dst.max()] = 255
    coordinates = np.argwhere(mask)

    return coordinates

In [312]:
"""Function finds upper points of boots. If there are no boots, take lower point of legs"""
def get_boot_corners(height, width, boot, leg):
    is_boot=False
    if np.mean(boot) == 0:
        boot = leg
        edge_point = np.array([0,0])
    else:
        edge_point = np.array([height,width])
        is_boot = True

    corners = get_corners(boot)
    for point in corners:
        corner = np.flip(point)
        if is_boot:
            if corner[0] < edge_point[1] and corner[1] < edge_point[0]:
                edge_point = corner
        else:
            if corner[1] > edge_point[0]:
                edge_point = corner
    return edge_point

In [313]:
"""This function outputs segmentation mask. Here are also heuristics and crutches against corner cases"""
def create_segment(directory, segmented_name):

    # read images from folders
    segmented = cv2.imread(rf"{directory}\human_parsing\{segmented_name}.png")
    ord_im = cv2.imread(rf"{directory}\image\{segmented_name}.jpg")

    # find height and width of image
    height = int(segmented.shape[0])
    width = int(segmented.shape[1])

    # get all needed types of segments
    pants = find_pants(segmented)
    left_leg = cv2.inRange(segmented, (0, 60, 127), (0, 65, 131))
    right_leg = cv2.inRange(segmented, (0, 60, 0), (0, 65, 0))
    socks = cv2.inRange(segmented, (0, 0, 60), (0, 0, 64))
    left_arm = cv2.inRange(segmented, (125, 125, 190), (131, 131, 195))
    right_arm = cv2.inRange(segmented, (125, 125, 60), (131, 131, 64))
    misc = cv2.inRange(255-segmented, (230, 230, 230), (240, 240, 240))
    left_boot = cv2.inRange(segmented, (0, 190, 120), (0, 195, 130))
    right_boot = cv2.inRange(segmented, (0, 190, 0), (0, 195, 0))


    # find upper, left lower and righ lower corners of pants
    pants_corners = get_corners(pants)
    upper = np.array([height,width])
    ld = np.array([height,0])
    rd = np.array([0,0])
    for pt in pants_corners:
        corner = np.flip(pt)
        if corner[0] < upper[0] and corner[1] < upper[1]:
            upper = corner
        elif corner[0] < ld[0] and corner[1] > ld[1]:
            ld = corner
        elif corner[0] > rd[0] and corner[1] > rd[1]:
            rd = corner

    # manually assign best upper corners
    if abs(upper[0]-ld[0]) < abs(upper[0]-rd[0]):
        ru = np.array([rd[0]-(abs(upper[0]-ld[0])), upper[1]])
        lu = upper
    else:
        lu = np.array([ld[0]+(abs(upper[0]-rd[0])), upper[1]])
        ru = upper

    # fix lower corners if they are too close to upper
    diff_ver = abs(ld[1]-rd[1])
    if diff_ver > (rd[1] - ru[1])*0.4 or diff_ver > (ld[1] - lu[1])*0.4:
        if ld[1]>rd[1]:
            rd[1] = ld[1]
        else:
            ld[1] = rd[1]

    # if upper corners are too close to each other then move them
    diff_hor = abs(rd[0]-ld[0])
    if abs(ru[0]-lu[0]) < diff_hor*0.3:
        lu[0] -= diff_hor*0.3
        ru[0] += diff_hor*0.3

    # find corners of boots (top) or legs (bottom)
    left_boot_top = get_boot_corners(height, width, left_boot, left_leg)
    right_boot_top = get_boot_corners(height, width, right_boot, right_leg)

    # if there are no legs and boots, draw a line limited by pants' lower corner
    if np.mean(left_boot_top) == 0:
        left_border = ld
        left_boot_top = ld
    else:
        left_border = build_line(ld, lu, left_boot_top[1])
    if np.mean(right_boot_top) == 0:
        right_border = rd
        right_boot_top = rd

    # draw a line from lower corner to boots/legs
    else:
        right_border = build_line(ru, rd, right_boot_top[1])

    # swap legs if they are crossed
    if left_boot_top[0] > right_boot_top[0]:
        left_boot_top, right_boot_top = right_boot_top, left_boot_top

    # many enhancements for complicated cases
    if left_border[0] > left_boot_top[0]:
        left_boot_top[0]-=50
        pts = np.array([ld, left_boot_top, right_boot_top, right_border, rd])
    elif right_border[0] < right_boot_top[0]:
        right_boot_top[0]+=50
        pts = np.array([ld, left_border, left_boot_top, right_boot_top, rd])
    else:
        if left_border[0] < 0:
            left_border[0] = round(abs(left_border[0]*0.5))
        if right_border[0] > width:
            right_border[0] -= round(right_border[0]*0.3)
        diff_left = left_boot_top[0] - left_border[0]
        diff_right = right_border[0] - right_boot_top[0]
        if diff_left>diff_right*1.7:
            left_border[0] += diff_right
        elif diff_left*1.7<diff_right:
            right_border[0] -= diff_left
        pts = np.array([ld, left_border, left_boot_top, right_boot_top, right_border, rd])

    # create mask
    poly_mask = cv2.fillPoly(np.zeros_like(segmented), [pts], (128,128,128))
    mask = cv2.inRange(poly_mask, (128, 128, 128), (128, 128, 128))+right_leg+left_leg+pants+socks-left_arm-right_arm-misc-left_boot-right_boot
    mask = cv2.inRange(mask, 250, 255)

    masked_ord_im = ord_im.copy()
    masked_ord_im[mask>0]=(128,128,128)

    return masked_ord_im

In [314]:
%%time
dataset_dir = f'{os.getcwd()}\dataset'

for image_name in os.listdir(f'{dataset_dir}\human_parsing'):

    image_name = image_name[:-4]
    cv2.imwrite(rf'{dataset_dir}\masked_images\{image_name}_masked.png', create_segment(dataset_dir, image_name))

CPU times: total: 47.2 s
Wall time: 47.8 s
